### 실습문제

<pre>
소방청 > 기관소개 > 전국소방기관 안내 에서
검색 박스에 '서울'을 입력하고 검색 버튼을 눌러서
서울시내 소재하는 100군데의 소방서, 안전센터를 찾아서
서울시 지도에 표시하세요.
</pre>

<pre>
1. (크롤링) 셀레니움으로 크롤링을 해서 소방서명, 주소, 전화번호  
   데이터를 가져와서 데이터프레임을 만든 후,
   데이터프레임에 새로운 컬럼 안전센터여부를 만들고 소방서는 0,
   안전센터는 1로 채우세요. (40점)
</pre>

In [53]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote
from tqdm.notebook import tqdm

In [54]:
chromedriver = 'chromedriver.exe'    # 상대 경로
driver = webdriver.Chrome(chromedriver)

In [55]:
base_url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=&myY=&searchKeyword='
url = base_url + quote('서울')
driver.get(url)
time.sleep(2)

In [56]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.area-location-list > ul > li')
len(lis)

10

In [57]:
# 소방서명
for i in range(len(lis)):
    title = lis[i].select_one('.title').get_text()
    print(title)

동작소방서
서대문소방서
광진소방서
송파소방서
양천소방서
은평소방서
노원소방서
종로소방서
도봉소방서
용산소방서


In [58]:
# 주소
li = lis[0]
address = li.select_one('address').get_text()
address

'서울특별시 동작구 여의대방로16길 55(신대방동)'

In [59]:
# 전화번호
tel = li.select_one('.tel').get_text()
tel

'02-847-1190'

In [60]:
# 페이지 클릭 1->2->다음페이지(3)->4->다음페이지(5)
# 1->2
driver.find_elements_by_css_selector('.pagination.pagination-small li')[3].click()
time.sleep(1)

# 2->다음페이지(3)
driver.find_elements_by_css_selector('.pagination.pagination-small li')[4].click()
time.sleep(1)

In [61]:
# 서울에 있는 100건의 소방기관을 데이터프레임화하기
# 페이지 넘기기는 주소로 넘깁니다.
chromedriver = '../05.Crawling/chromedriver.exe'
base_url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=&myY=&searchKeyword='
url = base_url + quote('서울')
lines = []
for i in tqdm(range(10)):   # 10page
    driver = webdriver.Chrome(chromedriver)
    driver.get(url + f'&pageIndex={i+1}')
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.area-location-list > ul > li')
    for li in lis:
        title = li.select_one('.title').get_text()
        address = li.select_one('address').get_text()
        tel = li.select_one('.tel').get_text()
        lines.append([title, address, tel])
    driver.close()

  0%|          | 0/10 [00:00<?, ?it/s]

In [62]:
# Check
len(df)

100

In [63]:
df = pd.DataFrame(lines, columns='소방서명 주소 전화번호'.split())
df.head(3)

,소방서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119


In [64]:
# 안전센터여부
df['안전센터여부'] = df.소방서명.apply(lambda x: 1 if x.find('안전센터') != -1 else 0)
print(df.head(2))
print(df.tail(2))

     소방서명                           주소          전화번호  안전센터여부
0   동작소방서  서울특별시 동작구 여의대방로16길 55(신대방동)   02-847-1190       0
1  서대문소방서      서울특별시 서대문구 연희로 182(연희동)  02-3144-1190       0
                 소방서명                         주소          전화번호  안전센터여부
98  송파소방서-거여-119 안전센터    서울특별시 송파구 마천로 329 (마천동)   02-400-0119       1
99  송파소방서-마천-119 안전센터  서울특별시 송파구 오금로51길 56 (마천동)  02-3401-2119       1


<pre>
2. (OpenAPI) 앞에서 구한 주소를 가지고, 카카오 로컬 Open API를
   이용하여 소방서의 위도, 경도 정보를 구한 후 데이터프레임에
   추가하세요. (30점)
</pre>

In [65]:
import requests
import warnings
warnings.filterwarnings('ignore')

In [66]:
# API Key 가져오기
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()
len(kakao_key)

32

In [67]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lng_list, lat_list = [], []
for i in df.index:
    try:
        url = f'{local_url}?query={quote(df.주소[i])}'
        header = {'Authorization': f'KakaoAK {kakao_key}'}
        result = requests.get(url, headers=header).json()
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        lng_list.append(0)
        lat_list.append(0)
        print(i)
df['위도'] = lat_list
df['경도'] = lng_list

30


In [68]:
df.head(31).tail(1)

,소방서명,주소,전화번호,안전센터여부,위도,경도
30,은평소방서-수색-119 안전센터,서울특별시 은평구 수색로 294 (수색동),02-376-0119,1,0.0,0.0


In [69]:
# index 30번 주소: 서울특별시 은평구 수색로 294 (수색동) 오류
# 주소 변경: 서울특별시 은평구 수색로 320
addr = '서울특별시 은평구 수색로 320'
url = f'{local_url}?query={quote(addr)}'
result = requests.get(url, headers=header).json()
df['위도'][30] = result['documents'][0]['y']
df['경도'][30] = result['documents'][0]['x']
print(df.위도[30], df.경도[30])

37.58523735699 126.889924404771


In [70]:
df.head(31).tail(3)

,소방서명,주소,전화번호,안전센터여부,위도,경도
28,서대문소방서-북가좌-119 안전센터,서울특별시 서대문구 증가로 242 (북가좌동),02-372-0119,1,37.582864,126.913004
29,서대문소방서-서대문_연희-119 안전센터,서울특별시 서대문구 연희로 182 (연희동),02-336-0119,1,37.573205,126.935996
30,은평소방서-수색-119 안전센터,서울특별시 은평구 수색로 294 (수색동),02-376-0119,1,37.585237,126.889924


<pre>
3. (지도시각화) 앞에서 구한 정보를 바탕으로 서울 지도(Open Street Map)에
   표시하되, 다음의 조건을 만족시키세요. (30점)
	- 툴팁에는 소방서명과 전화번호가 표시되어야 함
	- 팝업에는 소방서 주소가 가로로 표시되어야 함
	- 소방서와 안전센터의 아이콘이 다르게 표시되어야 함
	- 서울소재 소방서 위치라는 제목이 표시되어야 함
</pre>

In [71]:
import folium

In [76]:
def Safe_Center(i):
    if df.안전센터여부[i] == 1:
        return folium.Icon(color='green', icon='glyphicon glyphicon-leaf')
    else:
        return folium.Icon(color='red', icon='glyphicon glyphicon-fire')


In [77]:
map = folium.Map(
    location=[df.위도.mean(), df.경도.mean()], zoom_start=12
)

for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df.소방서명[i]} {df.전화번호[i]}',
        icon=Safe_Center(i)
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">서울소재 소방서 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map